In [0]:
dbutils.widgets.text(name="env", defaultValue="", label="Enter environment")
env = dbutils.widgets.get("env")
env

'dev'

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

In [0]:
%run "./paths"

('abfss://landing@dlsunitycat.dfs.core.windows.net/',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/bronze',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/silver',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/gold')

In [0]:
catalog = f'{env}_catalog'
upstream_table = f'{catalog}{silver_tables["plans"]}'
downstream_path = gold_paths['plans_dim_scd']
upstream_table, downstream_path

('dev_catalog.silver.plans_silver',
 'abfss://medallion@dlsunitycat.dfs.core.windows.net/gold/plans_dim_scd')

In [0]:
query = f"""
insert into delta.`{downstream_path}`
with lagged as (
select 
  plan_id,
  metal_level,
  lag(metal_level) over
    (partition by plan_id order by business_year) metal_level_prev,
  new_plan,
  lag(new_plan) over
    (partition by plan_id order by business_year) new_plan_prev,
  plan_type,
  lag(plan_type) over
    (partition by plan_id order by business_year) plan_type_prev,
    pandemic_era,
  lag(pandemic_era) over
    (partition by plan_id order by business_year) pandemic_era_prev,
  business_year
from {upstream_table}
),
changed as (
  select
    *,
    case
      when metal_level != metal_level_prev
      or metal_level_prev is null
      then 1
      when new_plan != new_plan_prev
      or new_plan_prev is null
      then 1
      when plan_type != plan_type_prev
      or plan_type_prev is null
      then 1
      when pandemic_era != pandemic_era_prev
      or pandemic_era_prev is null
      then 1
      else 0
    end did_change
  from lagged
),
changes as (
	select
		*,
		sum(did_change) over
			(partition by plan_id order by business_year) change_indicator
	from changed
)
select 
	plan_id,
	metal_level,
	new_plan,
    plan_type,
    pandemic_era,
	min(business_year) start_year,
	max(business_year) end_year,
	2024 current_year,
    CURRENT_TIMESTAMP() AS date_ingested
from changes
group by plan_id, change_indicator, metal_level, new_plan, plan_type, pandemic_era
"""


In [0]:
spark.sql(query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]